평가 최종 지표
1. faithfulness(ragas): 생성된 응답이 제공된 컨텍스트(문서 조각)에 기반한 사실과 일치하는지 평가
2. relevancy(ragas): 응답이 사용자의 질문과 관련되어 있는지 평가
3. context precision(정답지와 참고 문서의 관련성 비교)(ragas): 응답에 사용된 정보가 정확히 컨텍스트에서 추출된 정보인지, 즉 질문에 필요한 정보만 포함되었는지를 평가
    * 0.91
    * context precision@k
4. sentence Transformer(all-minilm-l6-v2): sentence-bert
    * 코사인 유사도 결과: 0.9
    * 정상적인 비교-판단: accuracy 0.74

In [28]:
import os
import pandas as pd

from pydantic import BaseModel, Field
from typing import List

from ragas.dataset_schema import SingleTurnSample
from ragas.metrics import Faithfulness, ResponseRelevancy, ContextPrecision

from langchain.chat_models import ChatOpenAI
from langchain_core.prompts import load_prompt
from langchain_core.output_parsers import PydanticOutputParser

In [29]:
dataset = pd.read_csv('../data/rag/rerank_dataset.csv')
dataset['reference_contexts'] = dataset['reference_contexts'].apply(lambda x : eval(x))
dataset['retrieved_contexts'] = dataset['retrieved_contexts'].apply(lambda x : eval(x))
dataset['cohere_contexts'] = dataset['cohere_contexts'].apply(lambda x : eval(x))
dataset['rerank_contexts'] = dataset['cohere_contexts'].apply(lambda x : '\n\n'.join(['document_{i}: \n{context}'.format(i=i, context=context) for i, context in enumerate(x[:9])]))

In [49]:
llm = ChatOpenAI(model='gpt-4o-mini', temperature=0.2)

prompt = load_prompt('../prompt/generate/generate.yaml')
chain = prompt | llm 

tmp = chain.invoke({'input_query': dataset.iloc[1]['user_input'], 'contexts': dataset.iloc[1]['cohere_contexts'][:9]})

print(dataset.iloc[1]['user_input'])
print(dataset.iloc[1]['reference'])
print(tmp.content)

스포츠 심리학자들은 선수의 성과 향상에 어떤 역할을 하며, 그들의 개인 문제에 대한 이해가 정신 훈련 프로그램의 효과성에 어떤 영향을 미치는가?
스포츠 심리학자들은 운동선수의 성과를 향상시키기 위해 여러 가지 역할을 수행합니다. 이들은 스포츠 심리학에서 지식과 이론을 창출하고 검증하기 위한 연구를 수행하고, 학교나 팀에서 관련 과목을 가르치며, 현장에서 멘탈 코치로서 심리학적 지식을 적용하여 성과를 개선합니다. 개인적인 문제에 대한 이해는 매우 중요합니다. 이러한 문제는 멘탈 훈련 프로그램의 효과성에 영향을 미칠 수 있기 때문입니다. 심리적 기술 훈련 중에는 성과와 관련된 개인적인 문제를 다루며, 이러한 개인적인 문제를 관리하는 것이 훈련이 운동선수의 성과에 긍정적인 영향을 미치도록 보장하는 데 중요합니다.
스포츠 심리학자들은 선수의 성과 향상을 위해 멘탈 코칭 프로그램을 개발하고 적용하며, 개인의 성격, 동기, 정서, 인지 특성과 운동 수행의 관련성을 탐구한다. 그들의 개인 문제에 대한 이해는 정신 훈련 프로그램의 효과성에 영향을 미치며, 특히 개인의 환경이나 상황도 중요한 변수로 작용하기 때문에 이러한 문제를 조절하고 관리하는 것이 필요하다.


In [50]:
data_samples = {
    'question': [dataset.iloc[1]['user_input']],
    'answer': [dataset.iloc[1]['reference']],
    'contexts': [dataset.iloc[1]['cohere_contexts'][:9]],
    'reference': [dataset.iloc[1]['reference']]
}

tmp_dataset = Dataset.from_dict(data_samples)
score = evaluate(tmp_dataset, metrics=[faithfulness, context_precision])

Evaluating: 100%|██████████| 2/2 [00:22<00:00, 11.32s/it]


In [51]:
score.to_pandas()

,user_input,retrieved_contexts,response,reference,faithfulness,context_precision
0,"스포츠 심리학자들은 선수의 성과 향상에 어떤 역할을 하며, 그들의 개인 문제에 대한...",[스포츠심리학을 연구하고 스포츠심리학의 연구 성과를 스포츠 현장에 적용하는\n스포츠...,스포츠 심리학자들은 운동선수의 성과를 향상시키기 위해 여러 가지 역할을 수행합니다....,스포츠 심리학자들은 운동선수의 성과를 향상시키기 위해 여러 가지 역할을 수행합니다....,1.0,1.0


In [30]:
llm = ChatOpenAI(model='gpt-4o-mini', temperature=0.2)

class GenerateAttributionOutput(BaseModel):
    attribution: List[str] = Field(..., description="List of attribution")
    answer: str = Field(..., description="Answer")

parser = PydanticOutputParser(pydantic_object=GenerateAttributionOutput)

prompt = load_prompt('../prompt/generate/generate_attribution.yaml')
prompt = prompt.partial(format=parser.get_format_instructions())

chain = prompt | llm 

In [31]:
tmp = chain.invoke({'input_query': dataset.iloc[0]['user_input'], 'contexts': dataset.iloc[0]['rerank_contexts']})

In [34]:
tmp_answer = parser.parse(tmp.content).answer
tmp_attribution = parser.parse(tmp.content).attribution

print(dataset.iloc[0]['user_input'])
print(dataset.iloc[0]['reference'])
print(tmp_answer)
print(tmp_attribution)

바벨 잡는 방법과 앉아받기 동작의 올바른 수행을 결합하여 최적의 리프팅 기술을 설명해줘.
바벨 잡는 방법에는 오버그립, 언더그립, 리버스그립, 훅그립이 있으며, 각각의 그립은 특정 운동에 적합하다. 예를 들어, 오버그립은 손바닥을 몸 쪽으로 향하게 하여 바벨을 잡는 방법으로, 일반적으로 많은 운동에서 사용된다. 앉아받기 동작은 바벨을 치골 또는 조금 높이 올린 후, 하방으로 다리를 강하게 옆으로 딛고 팔꿈치를 빠르게 앞으로 향하게 하여 가슴 위에 바벨을 위치시키는 것이 중요하다. 이 두 가지 기술을 결합하면, 바벨을 올바르게 잡고 앉아받기 동작을 통해 최적의 리프팅 성능을 발휘할 수 있다.
바벨을 잡는 방법과 앉아받기 동작의 올바른 수행을 결합하여 최적의 리프팅 기술을 설명하면 다음과 같다. 바벨을 잡을 때는 팔을 곧게 뻗고, 바벨은 수직으로 유지해야 하며, 몸통과 팔은 곧게 펴고 아래로 받기 동작을 실시해야 한다. 앉아받기 동작에서는 바벨을 치골 또는 조금 높이 올릴 때 하방으로 다리를 강하게 옆으로 딛고, 팔꿈치를 빠르게 앞으로 향하게 하여 가슴 위에 바벨을 빨리 위치시켜야 한다. 발은 빠르게 옆으로 옮겨 새로운 중심축을 만들고, 무릎은 제한된 범위까지 굽혀 대퇴부 후면을 종아리에 닿게 하며, 어깨는 수평을 유지하고 등은 자연스러운 아치형 상태에서 상체를 곧게 유지해야 한다.
['document_1', 'document_2', 'document_3', 'document_4', 'document_5']


In [45]:
dataset.iloc[0]['rerank_contexts']

"document_0: \n앉아받기란 잡아채기 동작을 실행한 다음 선수의 몸체를 바벨 아래로 밀어 넣으\n면서 바벨을 받치고 안정된 자세를 취하는 것을 말한다. 앉아받기 동작은 선수가\n발휘한 힘뿐만 아니라 선수와 바벨사이의 능동적 상호작용에 의해서 시작된다. 이\n동작에서 바벨의 움직임은 수직 상승한 다음 아래로 쳐진 올가미 형태를 보인다. 이 동작은 매우 빠르게 실시해야 하는데 그 이유는 팔을 뻗어 바를 바치는 동작을\n이루기까지 바벨이 몸의 중심과 많이 떨어져 있어 매우 불안한 상태이기 때문이다. 바를 바치는 안정된 동작으로 이동하기 위해서는 앉아받기 시 바벨을 최대한 높이 올려 몸체를 바벨 아래로 밀어 넣는 시간을\n충분히 갖는 것이 필요하다. 앉아받기 동작을 효율적으로 수행하기 위\n해서는 양다리를 벌릴 때 발뒤꿈치 부분을\n엉덩이 관절 수직 아래에 위치시켜 바벨을\n들기 위한 수직힘의 분산을 막아야 하며, 발\n끝은 약 45도 방향으로 벌려 기저면을 넓혀\n주어야한다.\n\ndocument_1: \n몸통 펴고 잡아채기 동작이 끝나고 바벨이 하지 신전력에 수직상방으로 올라가는\n단계이며, 매우 빠른 시간에 이루어지기 때문에 지도자들은 이 구간의 중요성을 인\n식 못할 수 있다. 이때는 폭발적인 하지 신전력에 의해바벨이 위로 던져진 상태이므\n로 무게를 느낄 수 없어야 잡아채기 동작을 바르게 수행한 것이다. 효과적인 앉아받\n기 동작을 하기 위해서〈그림 54〉에서 보는 것처럼 바벨이 최고의 정점에서 몸통\n쪽으로 가깝게 향하도록 팔의 움직임이 중요하며, 지면으로부터 점프를 하면서 하방\n으로 빠른 앉아 받기를 하는 기술동작이 필요하다. 인상기술 동작에서 잡아채기 후 앉아받기 동작을 효율적으로 수행하지 못하면 실패 확률이 높을 수 있으며, 바벨을\n앉아받기 동작에서 고정 시키더라도 무게중심이 높아 팔로 바벨을 정지시키기 위하\n여 과도한 힘을 사용하여 팔꿈치, 어깨, 손목의 상해가 발생할 수 있다.\n\ndocument_2: \n앉아받기 동작은 몸통 완전

In [35]:
dataset.iloc[0]['cohere_contexts'][:9]

['앉아받기란 잡아채기 동작을 실행한 다음 선수의 몸체를 바벨 아래로 밀어 넣으\n면서 바벨을 받치고 안정된 자세를 취하는 것을 말한다. 앉아받기 동작은 선수가\n발휘한 힘뿐만 아니라 선수와 바벨사이의 능동적 상호작용에 의해서 시작된다. 이\n동작에서 바벨의 움직임은 수직 상승한 다음 아래로 쳐진 올가미 형태를 보인다. 이 동작은 매우 빠르게 실시해야 하는데 그 이유는 팔을 뻗어 바를 바치는 동작을\n이루기까지 바벨이 몸의 중심과 많이 떨어져 있어 매우 불안한 상태이기 때문이다. 바를 바치는 안정된 동작으로 이동하기 위해서는 앉아받기 시 바벨을 최대한 높이 올려 몸체를 바벨 아래로 밀어 넣는 시간을\n충분히 갖는 것이 필요하다. 앉아받기 동작을 효율적으로 수행하기 위\n해서는 양다리를 벌릴 때 발뒤꿈치 부분을\n엉덩이 관절 수직 아래에 위치시켜 바벨을\n들기 위한 수직힘의 분산을 막아야 하며, 발\n끝은 약 45도 방향으로 벌려 기저면을 넓혀\n주어야한다.',
 '몸통 펴고 잡아채기 동작이 끝나고 바벨이 하지 신전력에 수직상방으로 올라가는\n단계이며, 매우 빠른 시간에 이루어지기 때문에 지도자들은 이 구간의 중요성을 인\n식 못할 수 있다. 이때는 폭발적인 하지 신전력에 의해바벨이 위로 던져진 상태이므\n로 무게를 느낄 수 없어야 잡아채기 동작을 바르게 수행한 것이다. 효과적인 앉아받\n기 동작을 하기 위해서〈그림 54〉에서 보는 것처럼 바벨이 최고의 정점에서 몸통\n쪽으로 가깝게 향하도록 팔의 움직임이 중요하며, 지면으로부터 점프를 하면서 하방\n으로 빠른 앉아 받기를 하는 기술동작이 필요하다. 인상기술 동작에서 잡아채기 후 앉아받기 동작을 효율적으로 수행하지 못하면 실패 확률이 높을 수 있으며, 바벨을\n앉아받기 동작에서 고정 시키더라도 무게중심이 높아 팔로 바벨을 정지시키기 위하\n여 과도한 힘을 사용하여 팔꿈치, 어깨, 손목의 상해가 발생할 수 있다.',
 '앉아받기 동작은 몸통 완전히 펴고 잡아채기 동작이후 빠른 앉아받기 자세를 하\n여야 한다. 완전

In [53]:
from datasets import Dataset
from ragas.metrics import faithfulness, context_precision
from ragas import evaluate

data_samples = {
    'question': [dataset.iloc[55]['user_input']],
    'answer': [dataset.iloc[55]['reference']],
    'contexts': [dataset.iloc[55]['cohere_contexts'][:9]],
    'reference': [dataset.iloc[55]['reference']]
}

tmp_dataset = Dataset.from_dict(data_samples)
score = evaluate(tmp_dataset, metrics=[faithfulness, context_precision])

Evaluating: 100%|██████████| 2/2 [00:25<00:00, 12.85s/it]


In [44]:
score.to_pandas()

,user_input,retrieved_contexts,response,reference,faithfulness,context_precision
0,바벨 잡는 방법과 앉아받기 동작의 올바른 수행을 결합하여 최적의 리프팅 기술을 설명해줘.,[앉아받기란 잡아채기 동작을 실행한 다음 선수의 몸체를 바벨 아래로 밀어 넣으\n면...,"바벨 잡는 방법에는 오버그립, 언더그립, 리버스그립, 훅그립이 있으며, 각각의 그립...","바벨 잡는 방법에는 오버그립, 언더그립, 리버스그립, 훅그립이 있으며, 각각의 그립...",0.333333,0.952629


In [64]:
type(dataset.iloc[0]['cohere_contexts'][:9])

list